In [6]:
using BenchmarkTools
using DRR

We will need to benchmark four experiments: 

|     | Serial     | Parallel     |
|-----|------------|--------------|
| CPU | CPU Serial | CPU Parallel |
| GPU | GPU Serial | GPU Parallel |

In [5]:
# Generate voxel grid

# Load the CT volume and convert it to a regular grid with correct spacing
ct_scan, ΔX, ΔY, ΔZ = read_dicom("../data/cxr/")
voxel_grid = volume2grid(ct_scan, ΔX, ΔY, ΔZ)

# Initialize the X-ray imaging system's geometry
camera = Camera(center=(-2,-2,-2))
detector = Detector(center=(2,2,2), normal=(-2,-2,-2), height=200, width=200)

LoadError: UndefVarError: read_dicom not defined

In [ ]:
# Generate the DRR
sampling_interval = 0.01
drr = make_drr(voxel_grid, camera, detector, sampling_rate)